In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Definition of a Curve
class Curve:
    # Constructor
    def __init__(self, list_of_points, closed=False):
        self.list_of_points = list_of_points
        self.closed = closed
        self.J = len(list_of_points)
    
    # Square Bracket Overload
    def __getitem__(self, key):
        return self.list_of_points[key % self.J]
    def __setitem__(self, key, value):
        self.list_of_points[key] = value
    
    # Length
    def __len__(self):
        return self.J
    
    # Curve Addition
    def __add__(self, otherCurve):
        if len(self) == len(otherCurve) and self.closed == otherCurve.closed:
            return Curve([self[i] + otherCurve[i] for i in range(len(self))])
    
    # Curve Length
    def curveLength(self):
        l = 0
        for i in range(self.J - 1):
            edgeLength = np.linalg.norm(self[i + 1] - self[i])
            l += edgeLength ** 2
        if self.closed:
            edgeLength = np.linalg.norm(self[-1] - self[0])
            l += edgeLength ** 2
        return l

In [ ]:
# p, q, T are all array
def kernelalphabeta(p, q, T, alpha=2, beta=4):
    pmq = p - q
    numerator = np.linalg.norm(np.cross(T, pmq)) ** alpha
    denominator = np.linalg.norm(pmq, 1) ** beta
    return numerator / denominator

# Pass a list of points.
def energy(points):
    J = len(points)

    e = 0
    for i in range(J):
        for j in range(J):
            if abs(i - j) > 1 and abs(i - j + J) > 1 and abs(i - j - J) > 1:
                xi = points[i % J]
                xipm = points[(i+1) % J]
                xj = points[j % J]
                xjpm = points[(j+1) % J]
                xI = xipm - xi
                lI = np.linalg.norm(xI)
                TI = xI / lI
                kernelAB = 0
                kernelAB += kernelalphabeta(xi, xj, TI) 
                kernelAB += kernelalphabeta(xi, xjpm, TI) 
                kernelAB += kernelalphabeta(xipm, xj, TI) 
                kernelAB += kernelalphabeta(xipm, xjpm, TI)
                kernelAB *= 0.25 * lI * np.linalg.norm(xjpm-xj)
                e += kernelAB
                print(f"{i}, {j}: kernelAB: {kernelAB}")

    return e


In [ ]:
# Generates the index pairs that are responsible for the derivative.
# Note that 4(J-1) pairs are generated.
def derivative_index(k, J):
    index_list = []
    for i in range(J):
        if abs(k - i) > 1 and abs(k - i + J) > 1 and abs(k - i - J):
            index_list.append((k, i))
    for i in range(J):
        if abs(k - 1 - i) > 1 and abs(k - 1 - i + J) > 1 and abs(k - 1 - i - J):
            index_list.append((k-1, i))
    for i in range(J):
        if abs(k - i) > 1 and abs(k - i + J) > 1 and abs(k - i - J):
            index_list.append((i, k))
    for i in range(J):
        if abs(k - 1 - i) > 1 and abs(k - 1 - i + J) > 1 and abs(k - 1 - i - J):
            index_list.append((i, k-1))
    

    return index_list

# Derivative of \xi_{k,j} with respect to x_k
def d_xi_kj(curve, k, j):
    kEdge = curve[k+1] - curve[k]
    kEdgeLength = np.linalg.norm(kEdge)
    kjEdge = curve[k] - curve[j]
    kjEdgeLength = np.linalg.norm(kjEdge)
    res = -2 * kEdge * (kjEdgeLength**2)
    res += 2 * (kEdgeLength**2) * kjEdge
    res += -2 * np.dot(kEdge, kjEdge) * (kEdge - kjEdge)
    return res

# Derivative of \eta_{k,j} with respect to x_k
def d_eta_kj(curve, k, j, alpha=2, beta=4):
    jkEdge = curve[k] - curve[j]
    jkEdgeLength = np.linalg.norm(jkEdge)
    kEdge = curve[k+1] - curve[k]
    kEdgeLength = np.linalg.norm(kEdge)
    res = beta * (jkEdgeLength ** (beta - 2)) * kEdgeLength**alpha * jkEdge
    res -= jkEdgeLength**beta * alpha * kEdgeLength**(alpha-2) * kEdge
    return res

# Derivative of \xi_{k-1, j} with respect to x_k
def d_xi_kmj(curve, k, j):
    kEdge = curve[k] - curve[k-1]
    kmjEdge = curve[k-1] - curve[j]
    kEdgeLength = np.linalg.norm(kEdge)
    kmjEdgeLength = np.linalg.norm(kmjEdge)
    res = 2 * (kmjEdgeLength**2) * kEdge
    res -= 2 * np.dot(kEdge, kmjEdge) * kmjEdge
    return res

# Derivative of \eta_{k-1, j} with respect to x_k
def d_eta_kmj(curve, k, j, alpha=2, beta=4):
    kEdge = curve[k] - curve[k-1]
    kmjEdge = curve[k-1] - curve[j]
    kEdgeLength = np.linalg.norm(kEdge)
    kmjEdgeLength = np.linalg.norm(kmjEdge)
    res = kmjEdgeLength**beta * alpha * kEdgeLength**(alpha - 2) * kEdge
    return res

# Derivative of \xi_{j, k} with respect to x_k
def d_xi_jk(curve, j, k):
    jEdge = curve[j+1] - curve[j]
    jEdgeLength = np.linalg.norm(jEdge)
    jkEdge = curve[k] - curve[j]
    res = 2 * jEdgeLength**2 * jkEdge
    res -= 2 * np.dot(jEdge, jkEdge) * jEdge
    return res

def d_eta_jk(curve, j, k, alpha=2, beta=4):
    jEdge = curve[j+1] - curve[j]
    jEdgeLength = np.linalg.norm(jEdge)
    jkEdge = curve[k] - curve[j]
    jkEdgeLength = np.linalg.norm(jkEdge)
    res = jEdgeLength**alpha * beta * jkEdgeLength**(beta-2) * jkEdge
    return res




#derivative_index(0, 8)
d_eta_jk(curve, 1, 3)

In [ ]:
points = []
resolution = 10
for i in range(resolution):
    theta = 2 * np.pi / resolution * i
    points.append(np.array([np.cos(theta), np.sin(theta), 0], dtype="float64"))
curve = Curve(points)
